In [1]:
import cv2
import numpy as np


In [2]:
width = 900
height = (int(width / 3)) * 2

img = np.ones((height, width, 3), np.uint8)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# img = cv2.imread("colorbar.jpg")
bgr_norm = img.astype(float) / 255.0
# 計算 K 通道
K = 1 - np.max(bgr_norm, axis=2)

# 防止除以0,設定一個最小閾值
denominator = 1 - K
denominator[denominator == 0] = 1e-10

# 計算 C、M、Y 通道
C = (1 - bgr_norm[..., 2] - K) / denominator
M = (1 - bgr_norm[..., 1] - K) / denominator
Y = (1 - bgr_norm[..., 0] - K) / denominator

# 歸一化回 0-255 並轉換為 uint8
cmyk = np.zeros(img.shape[:2] + (4,), dtype=np.uint8)
cmyk[..., 0] = (C * 255).astype(np.uint8)  # C 通道
cmyk[..., 1] = (M * 255).astype(np.uint8)  # M 通道
cmyk[..., 2] = (Y * 255).astype(np.uint8)  # Y 通道
cmyk[..., 3] = (K * 255).astype(np.uint8)  # K 通道


cv2.imshow("white", cmyk)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
red_cmyk = np.zeros(img.shape[:2] + (4,), dtype=np.uint8)
red_cmyk[..., 0] = (C * 0).astype(np.uint8)
red_cmyk[..., 1] = (M * 255).astype(np.uint8)
red_cmyk[..., 2] = (Y * 255).astype(np.uint8)
red_cmyk[..., 3] = (K * ((255 / 100) * 10)).astype(np.uint8)

cv2.imshow('red', red_cmyk)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
bg_cmyk = np.zeros(img.shape[:2] + (4,), dtype=np.uint8)
bg_cmyk[..., 0] = (C * 255).astype(np.uint8)
bg_cmyk[..., 1] = (M * ((255/100)*80)).astype(np.uint8)
bg_cmyk[..., 2] = (Y * 0).astype(np.uint8)
bg_cmyk[..., 3] = (K * ((255/100)*20)).astype(np.uint8)


cv2.imshow("blue_green", bg_cmyk)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
bg_src = np.ones((img.shape[0:2]/4) + (3,), np.uint8) * 255

TypeError: unsupported operand type(s) for /: 'tuple' and 'int'